In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import json
from ollama import Client
from IPython.display import display_markdown, display_latex

from src.tool import ToolKit, ToolCallProcessor
from functions import summation, multiplication

### Connect to local Ollama_server

In [14]:
client = Client(host="http://localhost:11434")

model_name = "qwen2.5:7b"

# Pull a model
result = client.pull(model=model_name)
result

{'status': 'success'}

### Registry functions

In [15]:
# Instanciar o registro de ferramentas
toolkit = ToolKit(tools=[summation, multiplication])

### Build the prompt

In [16]:
tools = "\n\n".join(
    [json.dumps(schema, indent=4) for schema in toolkit.tools_schemas()]
)


TOOL_PROMPT = (
    """
You are a function calling AI model.
If a function or tool is unavailable, responde with trained data.
You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags, here are the available tools:
<tools>
%s
</tools>

For each function call, to a listed function, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>, "id": <monotonically-increasing-id>}
</tool_call>
"""
    % tools
)

In [17]:
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_PROMPT,
    }
]

agent_chat_history = []

USER_PROMPT = "Can you multiple 25 by 12 and sum 3 with 6 and search what is an LLM on google.com?"

tool_chat_history.append({"role": "user", "content": USER_PROMPT})
agent_chat_history.append({"role": "user", "content": USER_PROMPT})

# Request tools action
tools_calls_request = client.chat(model=model_name, messages=tool_chat_history)
# Parse the response into model default format
tools_calls_str = tools_calls_request["message"]["content"]

tool_calls = ToolCallProcessor(tools_calls_str)

In [18]:
tool_calls

In [19]:
# Return the final response using the tools retrivied info
for call in tool_calls.tool_calls:
    try:
        result = toolkit.get_tool_by_name(call.name).run(call.arguments)

        agent_chat_history.append(
            {"role": "tool", "content": f"result of call to {call.name}: {result}"}
        )
    except AttributeError as ae:
        print(ae)

'NoneType' object has no attribute 'run'


In [20]:
tool_chat_history

[{'role': 'system',
  'content': '\nYou are a function calling AI model.\nIf a function or tool is unavailable, responde with trained data.\nYou may call one or more functions to assist with the user query.\nYou are provided with function signatures within <tools></tools> XML tags, here are the available tools:\n<tools>\n{\n    "name": "summation",\n    "doc": "\\n    Add two integers.\\n\\n    Parameters\\n    ----------\\n    x : int\\n        The first integer to be added.\\n    y : int, optional\\n        The second integer to be added (default is 0).\\n\\n    Returns\\n    -------\\n    int\\n        The sum of `x` and `y`.\\n    ",\n    "parameters": {\n        "properties": {\n            "x": {\n                "type": "int"\n            },\n            "y": {\n                "type": "int"\n            }\n        }\n    }\n}\n\n{\n    "name": "multiplication",\n    "doc": "\\n    Multiply two integers.\\n\\n    Parameters\\n    ----------\\n    x : int\\n        The first inte

In [21]:
agent_chat_history

[{'role': 'user',
  'content': 'Can you multiple 25 by 12 and sum 3 with 6 and search what is an LLM on google.com?'},
 {'role': 'tool', 'content': 'result of call to multiplication: 300'},
 {'role': 'tool', 'content': 'result of call to summation: 9'}]

In [22]:
tool_response = client.chat(model=model_name, messages=agent_chat_history)

response_using_tool = tool_response["message"]["content"]

display_markdown(response_using_tool, raw=True)

Sure, let's break it down:

1. Multiplying 25 by 12 gives us 300.
2. Adding 3 and 6 together gives us a sum of 9.

Now, if you're asking me to search for "LLM" on Google, I can provide information based on common understanding or general knowledge, but I won't actually perform an internet search in real-time. An LLM typically stands for Large Language Model, which is a type of artificial intelligence model used in natural language processing tasks such as text generation and translation.

Would you like more details about LLMs or anything else?